In [1]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import  metrics   #Additional scklearn functions
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection 
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
import plotly.express as px
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

In [17]:
df = pd.read_csv("Training.csv")
df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


**Shape of dataframe**

In [18]:
print("Training shape ",df.shape)

Training shape  (4920, 133)


**Checking for null values**

In [19]:
df.isnull().sum().sum()

0

**checking value count for each column**

In [20]:
for name in df.columns:
  print(name)
  print(df[name].value_counts(),end="\n\n----------")

itching
0    4242
1     678
Name: itching, dtype: int64

----------skin_rash
0    4134
1     786
Name: skin_rash, dtype: int64

----------nodal_skin_eruptions
0    4812
1     108
Name: nodal_skin_eruptions, dtype: int64

----------continuous_sneezing
0    4698
1     222
Name: continuous_sneezing, dtype: int64

----------shivering
0    4812
1     108
Name: shivering, dtype: int64

----------chills
0    4122
1     798
Name: chills, dtype: int64

----------joint_pain
0    4236
1     684
Name: joint_pain, dtype: int64

----------stomach_pain
0    4698
1     222
Name: stomach_pain, dtype: int64

----------acidity
0    4698
1     222
Name: acidity, dtype: int64

----------ulcers_on_tongue
0    4812
1     108
Name: ulcers_on_tongue, dtype: int64

----------muscle_wasting
0    4812
1     108
Name: muscle_wasting, dtype: int64

----------vomiting
0    3006
1    1914
Name: vomiting, dtype: int64

----------burning_micturition
0    4704
1     216
Name: burning_micturition, dtype: int64

---------

**correlation table**

In [11]:
fig = px.imshow(df.corr(),text_auto=True)
fig.show()

In [25]:
class config:
    NUM_FOLDS = 10
    SEED = 541
    TARGET = 'prognosis'
    JUNK = -1

def create_folds(data):    
    data["kfold"] = -1

    kf = model_selection.StratifiedKFold(n_splits=config.NUM_FOLDS, shuffle=True, random_state=config.SEED)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data[config.TARGET].values)):
        data.loc[v_, 'kfold'] = f    
    
    return data

In [26]:
def retrieve_data(train_folds,fold):


  train_data = train_folds[train_folds['kfold'] != fold].reset_index(drop=True)
  valid_data = train_folds[train_folds['kfold'] == fold].reset_index(drop=True)

  x_train = train_data.drop(['kfold', config.TARGET], axis=1)
  cols = x_train.columns
  x_train = x_train.values
  y_train = train_data[config.TARGET].values
      
  x_valid = valid_data.drop(['kfold', config.TARGET], axis=1).values
  y_valid = valid_data[config.TARGET].values


  return x_train,x_valid,y_train,y_valid

In [27]:
train1 = df
train_folds = create_folds(train1)

**logistic regression**

In [28]:
max = 0
lr_dummy = None
for fold in range(config.NUM_FOLDS):
  
  x_train,x_valid,y_train,y_valid = retrieve_data(train_folds,fold)
  lr = LogisticRegression(multi_class='ovr',max_iter=400)

  lr.fit(x_train, y_train)


  y_pred_train = lr.predict(x_train)  #np.where(yhat_train[:,1] >= best_thresh,1,0)
  y_pred_test = lr.predict(x_valid) #np.where(yhat_test[:,1] >= best_thresh,1,0)
  
  if(accuracy_score(y_valid,y_pred_test) > max):
    max = accuracy_score(y_valid,y_pred_test)
    lr_dummy = lr


  print("Training accuracy : ",accuracy_score(y_train,y_pred_train))
  print("Testing accuracy : ",accuracy_score(y_valid,y_pred_test))
  # #print("F1_Score : ",f1_score(y_valid, y_pred_test))
  # print("Precision score : ",precision_score(y_valid, y_pred_test))
  # print("Recall score : ",recall_score(y_valid, y_pred_test))
  print(classification_report(y_valid, y_pred_test))
  print(confusion_matrix(y_valid, y_pred_test))

  print("\n\n-----------------------------------------------------------------------",end="\n\n")

Training accuracy :  1.0
Testing accuracy :  1.0
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        12
                                   AIDS       1.00      1.00      1.00        12
                                   Acne       1.00      1.00      1.00        12
                    Alcoholic hepatitis       1.00      1.00      1.00        12
                                Allergy       1.00      1.00      1.00        12
                              Arthritis       1.00      1.00      1.00        12
                       Bronchial Asthma       1.00      1.00      1.00        12
                   Cervical spondylosis       1.00      1.00      1.00        12
                            Chicken pox       1.00      1.00      1.00        12
                    Chronic cholestasis       1.00      1.00      1.00        12
                            Common Cold       1.00      1.0

In [29]:
# save the model to disk
import pickle
filename = 'lr_model.pkl'
pickle.dump(lr_dummy, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

**Decision Tree**

In [33]:
max = 0
dt_dummy = None
for fold in range(config.NUM_FOLDS):
  
  x_train,x_valid,y_train,y_valid = retrieve_data(train_folds,fold)
  dt = DecisionTreeClassifier(max_depth=60)

  dt.fit(x_train, y_train)


  y_pred_train = dt.predict(x_train)  #np.where(yhat_train[:,1] >= best_thresh,1,0)
  y_pred_test = dt.predict(x_valid) #np.where(yhat_test[:,1] >= best_thresh,1,0)
  
  if(accuracy_score(y_valid,y_pred_test) > max):
    max = accuracy_score(y_valid,y_pred_test)
    dt_dummy = dt


  print("Training accuracy : ",accuracy_score(y_train,y_pred_train))
  print("Testing accuracy : ",accuracy_score(y_valid,y_pred_test))
  print(classification_report(y_valid, y_pred_test))
  print(confusion_matrix(y_valid, y_pred_test))

  print("\n\n-----------------------------------------------------------------------",end="\n\n")

Training accuracy :  0.9993224932249323
Testing accuracy :  0.9939024390243902
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        12
                                   AIDS       1.00      1.00      1.00        12
                                   Acne       1.00      1.00      1.00        12
                    Alcoholic hepatitis       0.80      1.00      0.89        12
                                Allergy       1.00      1.00      1.00        12
                              Arthritis       1.00      1.00      1.00        12
                       Bronchial Asthma       1.00      1.00      1.00        12
                   Cervical spondylosis       1.00      1.00      1.00        12
                            Chicken pox       1.00      1.00      1.00        12
                    Chronic cholestasis       1.00      1.00      1.00        12
                            C

In [34]:
filename = 'dt_model.pkl'
pickle.dump(dt_dummy, open(filename, 'wb'))

**Random forest**

In [35]:
max = 0
rf_dummy = None
for fold in range(config.NUM_FOLDS):
  
  x_train,x_valid,y_train,y_valid = retrieve_data(train_folds,fold)
  rf = RandomForestClassifier(n_estimators=400,max_depth=64)

  rf.fit(x_train, y_train)


  y_pred_train = rf.predict(x_train)  #np.where(yhat_train[:,1] >= best_thresh,1,0)
  y_pred_test = rf.predict(x_valid) #np.where(yhat_test[:,1] >= best_thresh,1,0)
  
  if(accuracy_score(y_valid,y_pred_test) > max):
    max = accuracy_score(y_valid,y_pred_test)
    rf_dummy = rf


  print("Training accuracy : ",accuracy_score(y_train,y_pred_train))
  print("Testing accuracy : ",accuracy_score(y_valid,y_pred_test))
  print(classification_report(y_valid, y_pred_test))
  print(confusion_matrix(y_valid, y_pred_test))

  print("\n\n-----------------------------------------------------------------------",end="\n\n")

Training accuracy :  1.0
Testing accuracy :  1.0
                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00        12
                                   AIDS       1.00      1.00      1.00        12
                                   Acne       1.00      1.00      1.00        12
                    Alcoholic hepatitis       1.00      1.00      1.00        12
                                Allergy       1.00      1.00      1.00        12
                              Arthritis       1.00      1.00      1.00        12
                       Bronchial Asthma       1.00      1.00      1.00        12
                   Cervical spondylosis       1.00      1.00      1.00        12
                            Chicken pox       1.00      1.00      1.00        12
                    Chronic cholestasis       1.00      1.00      1.00        12
                            Common Cold       1.00      1.0

In [36]:
filename = 'rf_model.pkl'
pickle.dump(rf_dummy, open(filename, 'wb'))